<a href="https://colab.research.google.com/github/edwinchenyj/scientific-computing-notes/blob/main/preconditioner/preconditioner_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preconditioner Experiements
First we need to setup a enviornment to test preconditioners. List of things we need:
* Random SPD matrix
  - [x] solve with CG
  - [x] solve with PCG using diagonal preconditioner
  - [ ] solve with PCG using incomplete cholesky
We can also test with SPD matrix from other applications. Notice that optimization problems with second-order methods almost always requires solving SPD matrices.


## Example 1: Random SPD matrix

In [108]:
import numpy as np
from scipy import stats
from scipy.sparse import csc_matrix
from scipy.sparse import spdiags, issparse, random, eye
from scipy.sparse.linalg import cg, LinearOperator, spsolve, spilu
rng = np.random.default_rng()
rvs = stats.poisson(25, loc=10).rvs
n = 1000
A = random(n,n, density=0.05, random_state=rng, data_rvs=rvs)
A = A + A.transpose() + n*eye(n)
b = np.random.rand(n,)

n_loops = 10

### scipy.sparse.spsolve

In [109]:
%%time 
for i in range(n_loops):
  x = spsolve(A, b)
print(np.linalg.norm(A.dot(x)-b))

1.9020110746891997e-14
CPU times: user 2.66 s, sys: 1.62 s, total: 4.28 s
Wall time: 2.24 s


### scipy.sparse.cg w/o preconditioner

In [110]:
%%time
for i in range(n_loops):
  x, info = cg(A,b)

print(info)
print(np.linalg.norm(A.dot(x)-b))

0
0.00010049117014895657
CPU times: user 42.2 ms, sys: 0 ns, total: 42.2 ms
Wall time: 46.5 ms


### scipy.sparse.cg w/ diagonal preconditioner

In [111]:
M_column = np.sqrt(np.diag(A.todense()))
M = spdiags(M_column,0,n,n)

In [112]:
%%time
for i in range(n_loops):
  x, info = cg(A,b,M=M)
print(info)
print(np.linalg.norm(A.dot(x)-b))

0
0.0001009970122341842
CPU times: user 41.9 ms, sys: 0 ns, total: 41.9 ms
Wall time: 46 ms
